In [ ]:
import numpy as np
from PIL import Image
import random

import torch
from torchvision import transforms as T
from torchvision.transforms import functional as F
import torchvision.transforms as transforms
from torchvision.datasets import Cityscapes

import augmentations as A

In [ ]:
# Define a list of transformations
augment_tranmforms = [A.Resize((256, 256)), # This resize is to get a reference when cropping
                    A.RandomHorizontalFlip(),
                    A.RandomCropWithProbability(220, 0.9),
                    A.RandomRotation(degrees=(-35, 35)),
                    A.Resize((256, 256)), # this resize is to make sure that all the output images have intened size
                    A.ToTensor()]

# Instanciate the Compose class with the list of transformations
data_transforms = A.Compose(augment_tranmforms)

dataset_path = 'C:\\Users\\jakub\\Desktop\\TUe\\AIES\\Q3\\5LSM0-Neural-networks-for-computer-vision\\FinalAssignment\\archive'

# Augmenting the images and mask at the same time
# Create transformed and AUGMENTED train dataset
augment_train_dataset = Cityscapes(dataset_path, split='train', mode='fine',
                            target_type='semantic', transforms=data_transforms)

In [ ]:
import matplotlib.pyplot as plt

N = 10 # munber of sampels to visualize
# Get the first three samples from the training dataset
samples = [augment_train_dataset[i] for i in range(N)]

# Create a 3x2 subplot grid
fig, axs = plt.subplots(N, 2, figsize=(10, 3*N))

for i, (img, mask) in enumerate(samples):
    # The images and masks are PyTorch tensors, so we need to convert them to numpy arrays for visualization
    img = img.permute(1, 2, 0).numpy()
    mask = mask.squeeze().numpy()

    # Plot the image in the first column
    axs[i, 0].imshow(img)
    axs[i, 0].axis('off')
    axs[i, 0].set_title('RGB Image -- Size:' + str(img.shape))

    # Plot the mask in the second column
    axs[i, 1].imshow(mask)
    axs[i, 1].axis('off')
    axs[i, 1].set_title('Target -- Size:' + str(mask.shape))

plt.tight_layout()
plt.show()

In [ ]:
# Define the transformations
data_transforms = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL Image to PyTorch Tensor
    transforms.Resize((256,256))
])

# Define a list of transformations
aug_tranmforms = [A.Resize((256, 256)), # This resize is to get a reference when cropping
                    A.RandomHorizontalFlip(),
                    A.RandomCropWithProbability(220, 0.9),
                    A.RandomRotation(degrees=(-35, 35)),
                    A.Resize((256, 256)), # this resize is to make sure that all the output images have intened size
                    A.ToTensor()]

# Instanciate the Compose class with the list of transformations
augment_transforms = A.Compose(aug_tranmforms)

# Create transformed train dataset
training_dataset = Cityscapes(dataset_path, split='train', mode='fine', target_type='semantic', transform=data_transforms, target_transform=data_transforms)
# training_dataset = Cityscapes(dataset_path, split='train', mode='fine', target_type='semantic', transforms=data_transforms)

print(f"Number of samples in the training dataset: {len(training_dataset)}")

# Create augmented train dataset
# augmented_dataset = Cityscapes(dataset_path, split='train', mode='fine', target_type='semantic', transform=augment_tranmforms, target_transform=augment_transforms)
augmented_dataset = Cityscapes(dataset_path, split='train', mode='fine',
                            target_type='semantic', transforms=augment_transforms)

print(f"Number of samples in the augmented dataset: {len(augmented_dataset)}")


In [ ]:
# Get the first item from the dataset
item = training_dataset[0]

# Print the type of the item
print(f'Type: {type(item)}')

img, mask = item

# If the item is a tuple, print the type and size of its elements
if isinstance(item, tuple):
    print(f'First element type (image): {type(img)}, size: {img.shape}')
    print(f'Second element type (mask): {type(mask)}, size: {mask.shape}')

In [ ]:
augment_item= augmented_dataset[0]

# Print the type of the item
print(f'Type: {type(augment_item)}')

augment_img, augment_mask = augment_item

# If the item is a tuple, print the type and size of its elements
if isinstance(augment_item, tuple):
    print(f'First element type (image): {type(augment_img)}, size: {augment_img.shape}')
    print(f'Second element type (mask): {type(augment_mask)}, size: {augment_mask.shape}')
    

In [ ]:
from torch.utils.data import ConcatDataset, DataLoader, random_split

# Combine the datasets
combined_dataset = ConcatDataset([training_dataset, augmented_dataset])

print(f"Number of samples in the combined dataset: {len(combined_dataset)}")

# Determine the lengths of the training and validation sets
total_size = len(combined_dataset)
train_size = int(0.8 * total_size)  # 80% for training
val_size = total_size - train_size  # 20% for validation

# Shuffle and Split the combined dataset 
train_dataset, val_dataset = random_split(combined_dataset, [train_size, val_size])

# Create the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, 
                                pin_memory=True if torch.cuda.is_available() else False)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False, 
                                pin_memory=True if torch.cuda.is_available() else False)

In [ ]:
# cheching the content of the dataloader
# For the train dataloader
for i, (inputs, targets) in enumerate(train_dataloader):
    print(f"In batch {i}, the type of inputs is {type(inputs)} and the type of targets is {type(targets)}")
    break  # We break after the first batch as we just want to know the type

# For the validation dataloader
for i, (inputs, targets) in enumerate(val_dataloader):
    print(f"In batch {i}, the type of inputs is {type(inputs)} and the type of targets is {type(targets)}")
    break  # We break after the first batch as we just want to know the type

In [ ]:
import matplotlib.pyplot as plt

N = 10 # number of sampels

# Get the first three samples from the training dataset
samples = [train_dataset[i] for i in range(N)]

# Create a 3x2 subplot grid
fig, axs = plt.subplots(N, 2, figsize=(10, 3*N))

for i, (img, mask) in enumerate(samples):
    # The images and masks are PyTorch tensors, so we need to convert them to numpy arrays for visualization
    img = img.permute(1, 2, 0).numpy()
    mask = mask.squeeze().numpy()

    # Plot the image in the first column
    axs[i, 0].imshow(img)
    axs[i, 0].axis('off')
    axs[i, 0].set_title('RGB Image')

    # Plot the mask in the second column
    axs[i, 1].imshow(mask)
    axs[i, 1].axis('off')
    axs[i, 1].set_title('Semantic Segmentation')

plt.tight_layout()
plt.show()

In [ ]:
import torch
import torch.nn as nn


class DiceLoss(nn.Module):
    def __init__(self, ignore_index=255, log_loss=False):
        super(DiceLoss, self).__init__()
        self.ingore_index = ignore_index
        self.log_loss = log_loss

    def forward(self, input, target):
        smooth = 1.

        # Apply softmax to input (model output)
        input = torch.softmax(input, dim=1)

        dice_loss = 0.

        print(input.size)

        # print('shape of input: ', input.size(1))
        for class_index in range(input.size(1)):
            valid = (target != self.ingore_index)
            # print(f'shape of valid: {valid}')
            # print(f'shape of input: {input.shape}')
            # print(f'shape of target: {target.shape}')
            input_flat = input[:, class_index, :, :][valid].contiguous().view(-1)
            target_flat = (target == class_index)[valid].contiguous().view(-1) # binary target for class_index

            intersection = (input_flat * target_flat).sum()

            class_loss = 1 - ((2. * intersection + smooth) / (input_flat.sum() + target_flat.sum() + smooth))
            dice_loss += class_loss

            print(f"Loss for class {class_index}: {class_loss.item()}")
            print(f"Aggregated loss: {dice_loss.item()}")

        mean_dice = dice_loss/input.size(1) # average loss over all classes
        print(f"Mean Dice Loss (avg. loss for all classes): {mean_dice.item()}")

        if self.log_loss:
            mean_dice = -torch.log(mean_dice)

        return mean_dice


In [ ]:
from model import Model
import utils

import torch.optim as optim
import losses as L

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instanciate Model()
model = Model()

# Specify the checkpoint file path
checkpoint_path = 'C:\\Users\\jakub\\Desktop\\TUe\\AIES\\Q3\\5LSM0-Neural-networks-for-computer-vision\\FinalAssignment\\model_checkpoints\\model_checkpoint_epoch_60_Mar14.pth'
device = torch.device('cpu') 

checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint)
criterion = DiceLoss(ignore_index=255, log_loss=True)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# VALIDATION
# Initialize accuracy variables for each epoch
total_correct_train, total_samples_train = 0, 0
total_correct_val, total_samples_val = 0, 0
# TRAINING
model.train()
running_train_loss = 0.0
running_val_loss = 0.0

for val_inputs, val_masks in train_dataloader:
    # Move inputs and masks to the GPU
    val_inputs, val_masks = val_inputs.to(device), val_masks.to(device)
    val_outputs = model(val_inputs)
    print(val_outputs.shape)
    val_masks = (val_masks*255).long().squeeze()     #*255 because the id are normalized between 0-1
    val_masks = utils.map_id_to_train_id(val_masks).to(device)
    val_loss = criterion(val_outputs, val_masks)
    running_val_loss += val_loss.item()
    # Compute validation accuracy
    _, val_predicted = torch.max(val_outputs, 1)
    total_correct_val += (val_predicted == val_masks.long().squeeze()).sum().item()
    total_samples_val += val_masks.numel()

    break
epoch_val_loss = running_val_loss
print(f"Validation epoch loss: {epoch_val_loss:.4f}")

In [ ]:
# # Testing the training lood with the validation loader and the augmented dataset
# from model import Model
# import model_executables as mex
# import losses as L

# import torch.optim as optim


# # Instanciate the model
# UNet_model = Model()

# # Move the model to the GPU if avaliable
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# UNet_model = UNet_model.to(device)

# criterion = L.DiceLoss(ignore_index=255, log_loss=True)
# optimizer = optim.Adam(UNet_model.parameters(), lr=0.01)

# # Train the instanciated model
# mex.train_model(UNet_model, train_dataloader, val_dataloader, num_epochs=2, patience=3, criterion=criterion, optimizer=optimizer)
